In [1]:
from torchvision import transforms
from PIL import Image

# # Define transformations
# transform = transforms.Compose([
#     transforms.Lambda(lambda img: img.convert('RGB')),  # Convert to RGB if needed
#     transforms.Resize((512, 512)),  # Resize images to a consistent size
#     transforms.ToTensor(),  # Convert images to PyTorch tensors
# ])

In [2]:
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim
import os
from pathlib import Path

In [3]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib'

In [4]:
cd anomalib/

d:\Shravtek\neutech_airfillter\anamolib\anomalib


In [5]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib\\anomalib'

In [6]:
# Import the datamodule
from anomalib.data import Folder

# Create the datamodule
datamodule = Folder(
    name="hazelnut_toy",
    root="datasets/hazelnut_toy",
    normal_dir="good",
    abnormal_dir="crack",
    task="classification",
    image_size=(512,512)
  )

# Setup the datamodule
datamodule.setup()

In [7]:
i, train_data = next(enumerate(datamodule.train_dataloader()))
print(train_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, val_data = next(enumerate(datamodule.val_dataloader()))
print(val_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, test_data = next(enumerate(datamodule.test_dataloader()))
print(test_data.keys())
# dict_keys(['image_path', 'label', 'image'])

dict_keys(['image_path', 'label', 'image'])
dict_keys(['image_path', 'label', 'image'])
dict_keys(['image_path', 'label', 'image'])


In [8]:
# Import the model and engine
from anomalib.models import Patchcore
from anomalib.engine import Engine

# Create the model and engine
model = Patchcore()
engine = Engine(task="classification",max_epochs=2)


In [9]:

# Train a Patchcore model on the given datamodule
engine.train(datamodule=datamodule, model=model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax                   | 0     
3 | image_threshold       | F1AdaptiveThreshold      | 0     
4 | pixel_threshold       | F1AdaptiveThreshold      | 0     
5 | image_metrics         | AnomalibMetricCollection | 0     
6 | pixel_metrics         | AnomalibMe

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │     0.800000011920929     │
└───────────────────────────┴───────────────────────────┘

## Validation


In [8]:
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Patchcore\hazelnut_toy\v2\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Patchcore\hazelnut_toy\v2\weights\lightning\model.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            1.0            │
└───────────────────────────┴───────────────────────────┘

In [9]:
test_results

[{'image_AUROC': 1.0, 'image_F1Score': 1.0}]

## OpenVINO Inference

Now that we trained and tested a model, we could check a single inference result using OpenVINO inferencer object. This will demonstrate how a trained model could be used for inference.


In [12]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if image.numel() == 0:
c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:61: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if crop_height > image_height or crop_width > image_width:
c:\Users\shan\.conda\envs\anamo\lib\site-packages\anomalib\data\transforms\center_crop.py:41: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore th

WindowsPath('D:/Shravtek/neutech_airfillter/anamolib/anomalib/results/Patchcore/hazelnut_toy/latest/weights/openvino/model.xml')

In [16]:
image_path = "D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/hazelnut_toy/crack/01.jpg"
image = read_image(path="D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/hazelnut_toy/crack/01.jpg")
plt.imshow(image)

In [19]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Patchcore\hazelnut_toy\latest


In [20]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [21]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

In [22]:
predictions = inferencer.predict(image=image_path)

In [23]:
print(predictions.pred_score, predictions.pred_label)

0.55394659682092 LabelName.ABNORMAL


In [27]:
# Visualize the original image
print(predictions)

ImageResult(image=[[[228 214  79]
  [231 217  82]
  [231 217  82]
  ...
  [226 213  82]
  [225 213  79]
  [229 217  83]]

 [[228 214  79]
  [228 214  79]
  [229 215  80]
  ...
  [226 214  80]
  [227 215  81]
  [228 216  80]]

 [[230 217  79]
  [230 217  79]
  [229 216  78]
  ...
  [225 213  79]
  [226 214  78]
  [222 210  74]]

 ...

 [[233 224  83]
  [232 223  82]
  [232 223  82]
  ...
  [223 211  73]
  [223 211  73]
  [222 210  72]]

 [[233 224  83]
  [229 220  79]
  [235 226  85]
  ...
  [225 214  73]
  [223 212  71]
  [224 213  72]]

 [[230 221  80]
  [230 221  80]
  [231 222  81]
  ...
  [219 208  67]
  [221 210  69]
  [226 215  74]]], pred_score=0.55394659682092, pred_label=1, anomaly_map=[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], gt_mask=None, gt_boxes=None, pred_boxes=None, box_labels=None, pred_mask=[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 